In [1]:
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy.sparse as sp
import math

from matplotlib.pyplot import savefig

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.optim import AdamW
import random

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp

In [2]:
training_flow = pd.read_csv('Insufficient data/training_flow.csv')
training_flow['Timestamp'] = pd.to_datetime(training_flow['Timestamp'])
training_flow.set_index('Timestamp', inplace=True)

training_speed = pd.read_csv('Insufficient data/training_speed.csv')
training_speed['Timestamp'] = pd.to_datetime(training_speed['Timestamp'])
training_speed.set_index('Timestamp', inplace=True)

test_flow = pd.read_csv('Insufficient data/test_flow.csv')
test_flow['Timestamp'] = pd.to_datetime(test_flow['Timestamp'])
test_flow.set_index('Timestamp', inplace=True)

test_speed = pd.read_csv('Insufficient data/test_speed.csv')
test_speed['Timestamp'] = pd.to_datetime(test_speed['Timestamp'])
test_speed.set_index('Timestamp', inplace=True)

num_link = training_flow.shape[1]

In [5]:
time_leg = 24
prediction_horizon = 1

In [6]:
continuous_training_flow = []
continuous_training_speed = []

continuous_training_time = []

training_times = training_flow.index

for i in range(len(training_times) - time_leg - prediction_horizon + 1):
    
    # Check if the 30 time points starting from the current line are consecutive
    if all(training_times[i + j] == training_times[i] + pd.Timedelta(minutes=5 * j) for j in range(1,time_leg + prediction_horizon)):
        
        # If consecutive, extract the data from these 30 lines
        data_slice_flow = training_flow.iloc[i:i + time_leg + prediction_horizon].values
        data_slice_speed= training_speed.iloc[i:i + time_leg + prediction_horizon].values
        
        # Add the data to the list
        continuous_training_flow.append(data_slice_flow)
        continuous_training_speed.append(data_slice_speed)
        
        continuous_training_time.append(training_times[i])
        
flow_data_training = np.array(continuous_training_flow)
speed_data_training = np.array(continuous_training_speed)

flow_data_training.shape

(176, 25, 150)

In [7]:
continuous_test_flow = []
continuous_test_speed = []

continuous_test_time = []

test_times = test_flow.index

for i in range(len(test_times) - time_leg - prediction_horizon + 1):
    
    # Check if the 30 time points starting from the current line are consecutive
    if all(test_times[i + j] == test_times[i] + pd.Timedelta(minutes=5 * j) for j in range(1,time_leg + prediction_horizon)):
        
        # If consecutive, extract the data from these 30 lines
        data_slice_flow = test_flow.iloc[i:i + time_leg + prediction_horizon].values
        data_slice_speed= test_speed.iloc[i:i + time_leg + prediction_horizon].values
        
        # Add the data to the list
        continuous_test_flow.append(data_slice_flow)
        continuous_test_speed.append(data_slice_speed)
        
        continuous_test_time.append(test_times[i])
        
flow_data_test = np.array(continuous_test_flow)
speed_data_test = np.array(continuous_test_speed)

flow_data_test.shape

(32, 25, 150)

In [8]:
training_set_q = torch.from_numpy(flow_data_training).to(torch.float32)
test_set_q = torch.from_numpy(flow_data_test).to(torch.float32)

training_set_v = torch.from_numpy(speed_data_training).to(torch.float32)
test_set_v = torch.from_numpy(speed_data_test).to(torch.float32)

print(training_set_q.shape)
print(test_set_q.shape)
print(training_set_v.shape)
print(test_set_v.shape)

torch.Size([176, 25, 150])
torch.Size([32, 25, 150])
torch.Size([176, 25, 150])
torch.Size([32, 25, 150])


In [9]:
v_f = pd.read_csv('parameter/v_f.csv').values
k_c = pd.read_csv('parameter/k_c.csv').values
mm = pd.read_csv('parameter/mm.csv').values

v_f = torch.from_numpy(v_f).to(torch.float32).reshape(-1)
k_c = torch.from_numpy(k_c).to(torch.float32).reshape(-1)
mm = torch.from_numpy(mm).to(torch.float32).reshape(-1)

v_f.shape

torch.Size([150])

==============================   Prediction model   =================================

In [12]:
adj = pd.read_csv('data/adj_matrix/proximity_matrix.csv').values
adj_correlation_q = pd.read_csv('data/adj_matrix/correlation_matrix_flow.csv').values
adj_knn_q = pd.read_csv('data/adj_matrix/knn_matrix_flow.csv').values
adj_correlation_v = pd.read_csv('data/adj_matrix/correlation_matrix_speed.csv').values
adj_knn_v = pd.read_csv('data/adj_matrix/knn_matrix_speed.csv').values

In [13]:
adj_correlation_q = torch.from_numpy(adj_correlation_q).to(torch.float32)
adj_correlation_v = torch.from_numpy(adj_correlation_v).to(torch.float32)

In [14]:
# Adjacency matrix normalization
def adj_def(adj):
   
    adj = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0
    r_mat_inv = sp.diags(r_inv)
    adj = r_mat_inv.dot(adj).A
    
    adj = torch.from_numpy(adj).to(torch.float32)

    return adj

In [15]:
adj_n = adj_def(adj)
print(adj_n.shape)
adj_knn_q = adj_def(adj_knn_q)
adj_knn_v = adj_def(adj_knn_v)

torch.Size([150, 150])


In [16]:
max_q = training_set_q.max(axis=0)[0].max(axis=0)[0]
min_q = training_set_q.min(axis=0)[0].min(axis=0)[0]

max_v = training_set_v.max(axis=0)[0].max(axis=0)[0]
min_v = training_set_v.min(axis=0)[0].min(axis=0)[0]

max_q.shape

torch.Size([150])

In [18]:
# Define the normalization function
def norm(x,max_x,min_x):
    x0 = 2*(x-min_x)/(max_x-min_x)-1
    return x0

In [19]:
# Define the inverse normalization function
def r_norm(x0,max_x,min_x):
    x = (x0+1) * (max_x-min_x)/2 + min_x
    return x

In [20]:
# Normalize flow and speed data
training_set_q0 = norm(training_set_q,max_q,min_q)
test_set_q0 = norm(test_set_q,max_q,min_q)

training_set_v0 = norm(training_set_v,max_v,min_v)
test_set_v0 = norm(test_set_v,max_v,min_v)

In [21]:
# Training set input and output
training_input_q = training_set_q0[:,:time_leg,:]
training_output_q = training_set_q0[:,time_leg:time_leg+prediction_horizon,:]

training_input_v = training_set_v0[:,:time_leg,:]
training_output_v = training_set_v0[:,time_leg:time_leg+prediction_horizon,:]

# Test set input and output
test_input_q = test_set_q0[:,:time_leg,:]
test_output_q = test_set_q0[:,time_leg:time_leg+prediction_horizon,:]

test_input_v = test_set_v0[:,:time_leg,:]
test_output_v = test_set_v0[:,time_leg:time_leg+prediction_horizon,:]

# Ground truth
training_true_q = training_set_q[:,time_leg:time_leg+prediction_horizon,:]
training_true_v = training_set_v[:,time_leg:time_leg+prediction_horizon,:]

test_true_q = test_set_q[:,time_leg:time_leg+prediction_horizon,:]
test_true_v = test_set_v[:,time_leg:time_leg+prediction_horizon,:]

In [22]:
num_link = training_set_q0.shape[2]
num_link

150

In [23]:
# Define the GraphConvolution layer
class GraphConvolution(Module):

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

# Define the LSTM unit
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, return_sequence):
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.return_sequence = return_sequence
        #self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)

    def forward(self, x):
        
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        output, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        if self.return_sequence==False:
            output = output[:,-1,:]
            output = output.contiguous().view(output.shape[0],-1)
        return output    

In [28]:
class GCN(Module):

    def __init__(self, nfea_gcn, nhid_gcn1, nhid_gcn2, nhid_gcn3, n_latent_gcn, num_link):
        super(GCN, self).__init__()
        
        self.nfea_gcn = nfea_gcn
        self.num_link = num_link
        
        self.nhid_gcn1 = nhid_gcn1
        self.nhid_gcn2 = nhid_gcn2
        self.nhid_gcn3 = nhid_gcn3
        
        self.gc11 = GraphConvolution(nfea_gcn, nhid_gcn1[0])
        self.gc12 = GraphConvolution(nhid_gcn1[0], nhid_gcn1[1])
        self.fc_n = nn.Linear(nhid_gcn1[2] * num_link, nhid_gcn1[2])
        
        
        self.gc21 = GraphConvolution(nfea_gcn, nhid_gcn2[0])
        self.gc22 = GraphConvolution(nhid_gcn2[0], nhid_gcn2[1])
        self.fc_m = nn.Linear(nhid_gcn2[2] * num_link, nhid_gcn2[2])
    
        self.gc31 = GraphConvolution(nfea_gcn, nhid_gcn3[0])
        self.gc32 = GraphConvolution(nhid_gcn3[0], nhid_gcn3[1])
        self.fc_s = nn.Linear(nhid_gcn3[2] * num_link, nhid_gcn3[2])
        
        self.fc = nn.Linear(nhid_gcn1[2], n_latent_gcn)
        
    def forward(self, x, adj_n, adj_c, adj_k):
        
        x1 = torch.tanh(self.gc11(x, adj_n))
        x1 = torch.tanh(self.gc12(x1, adj_n))
        x1 = x1.view(x1.shape[0],-1)
        x1 = torch.tanh(self.fc_n(x1))
        
        x2 = torch.tanh(self.gc21(x, adj_c))
        x2 = torch.tanh(self.gc22(x2, adj_c))
        x2 = x2.view(x2.shape[0],-1)
        x2 = torch.tanh(self.fc_m(x2))
        
        x3 = torch.tanh(self.gc31(x, adj_k))
        x3 = torch.tanh(self.gc32(x3, adj_k))
        x3 = x3.view(x3.shape[0],-1)
        x3 = torch.tanh(self.fc_s(x3))
        
        x0 = (x1 + x2 + x3)/3
        x0 = torch.tanh(self.fc(x0))
        
        return x0

In [29]:
class PIML(Module):
    
    def __init__(self, nfea_lstm, nfea_gcn, prediction_horizon, num_link,
                 nhid_lstm, nhid_gcn, n_latent_gcn, nhid_linear):
        super(PIML, self).__init__()
   
        self.nfea_lstm = nfea_lstm
        self.nfea_gcn = nfea_gcn
        self.prediction_horizon = prediction_horizon
        self.num_link = num_link
        self.nhid_linear = nhid_linear  

        # three-layer LSTM
        self.lstm1 = LSTM(nfea_lstm, nhid_lstm[0], return_sequence=True)
        self.lstm2 = LSTM(nhid_lstm[0], nhid_lstm[1], return_sequence=True)
        self.lstm3 = LSTM(nhid_lstm[1], nhid_lstm[2], return_sequence=False)
        
        # MGCN
        self.MGCN = GCN(nfea_gcn, nhid_gcn[0], nhid_gcn[1], nhid_gcn[2], n_latent_gcn, num_link)
        
        self.fc1 = nn.Linear(nhid_lstm[-1]+n_latent_gcn, nhid_linear[0])
        self.fc2 = nn.Linear(nhid_linear[0], prediction_horizon*nfea_lstm)
        
    def forward(self, fea, adj_n, adj_c, adj_k):
        
        x_t = self.lstm1(fea)
        x_t = self.lstm2(x_t)
        x_t = self.lstm3(x_t)

        x_s = fea.transpose(1,2)
        x_s = self.MGCN(x_s, adj_n, adj_c, adj_k)        

        x = torch.cat((x_t, x_s),dim=1)

        x = torch.tanh(self.fc1(x)) 
        x = torch.tanh(self.fc2(x))
        
        x = x.view(x.shape[0], self.prediction_horizon, self.nfea_lstm)

        return x

In [31]:
piml_q=PIML(nfea_lstm=num_link, nfea_gcn=time_leg, prediction_horizon=prediction_horizon,num_link = num_link,
            nhid_lstm=[512,512,512], nhid_gcn=[[32,32,32],[32,32,32],[32,32,32]], n_latent_gcn=512, nhid_linear=[512])
piml_v=PIML(nfea_lstm=num_link, nfea_gcn=time_leg, prediction_horizon=prediction_horizon,num_link = num_link,
            nhid_lstm=[512,512,512], nhid_gcn=[[32,32,32],[32,32,32],[32,32,32]], n_latent_gcn=512, nhid_linear=[512])

# Loss function
nn_loss = nn.MSELoss()
# Optimizers
optimizer = AdamW([
    {'params': piml_q.parameters(), 'lr': 0.001, 'weight_decay': 1e-2},
    {'params': piml_v.parameters(), 'lr': 0.001, 'weight_decay': 1e-2}
])
n_epochs = 2000
batch_size = 16
sample_size = training_set_q.shape[0]

In [32]:
def train(n_epochs, batch_size, sample_size):
    t0 = time.time()
    for epoch in range(n_epochs):
        
        idx = random.sample(range(0, sample_size),batch_size)
        
        input_q = training_input_q[idx]
        output_q = training_output_q[idx]
        
        input_v = training_input_v[idx]
        output_v = training_output_v[idx]
        
        optimizer.zero_grad()
        
        pred_q0 = piml_q(input_q, adj_n, adj_correlation_q, adj_knn_q)
        pred_v0 = piml_v(input_v, adj_n, adj_correlation_v, adj_knn_v)
        
        loss_q = nn_loss(output_q, pred_q0)
        loss_v = nn_loss(output_v, pred_v0)
        
        # S3
        pred_q = r_norm(pred_q0, max_q, min_q)
        pred_v = r_norm(pred_v0, max_v, min_v)
        v_cal = v_f / (1 + (pred_q/(pred_v*k_c))**mm) ** (2/mm)
        loss_fd_target = Variable(torch.Tensor(v_cal.shape).fill_(1.0), requires_grad=False)
        loss_fd = nn_loss(v_cal/pred_v, loss_fd_target)
        
        total_loss = loss_q + loss_v + 0.2 * loss_fd
            
        total_loss.backward()
        optimizer.step()
        
        print(
            "[Epoch %d/%d] [loss_q: %f] [loss_v: %f] [loss_fd: %f] [total_loss: %f] [time: %f]"
            % (epoch, n_epochs, loss_q.item(), loss_v.item(), loss_fd.item(), total_loss.item(), time.time()-t0)
        )

In [33]:
train(n_epochs = n_epochs, batch_size = batch_size, sample_size = sample_size)

[Epoch 0/2000] [loss_q: 0.273668] [loss_v: 0.353847] [loss_fd: 0.060086] [total_loss: 0.639533] [time: 0.442286]
[Epoch 1/2000] [loss_q: 0.254378] [loss_v: 0.355859] [loss_fd: 0.045873] [total_loss: 0.619411] [time: 0.712478]
[Epoch 2/2000] [loss_q: 0.214203] [loss_v: 0.227412] [loss_fd: 0.025441] [total_loss: 0.446704] [time: 0.991676]
[Epoch 3/2000] [loss_q: 0.150031] [loss_v: 0.195316] [loss_fd: 0.012953] [total_loss: 0.347938] [time: 1.259710]
[Epoch 4/2000] [loss_q: 0.120610] [loss_v: 0.196260] [loss_fd: 0.008886] [total_loss: 0.318647] [time: 1.480868]
[Epoch 5/2000] [loss_q: 0.094520] [loss_v: 0.142467] [loss_fd: 0.012162] [total_loss: 0.239419] [time: 1.698023]
[Epoch 6/2000] [loss_q: 0.087945] [loss_v: 0.102406] [loss_fd: 0.015163] [total_loss: 0.193384] [time: 1.911230]
[Epoch 7/2000] [loss_q: 0.088694] [loss_v: 0.097929] [loss_fd: 0.016326] [total_loss: 0.189888] [time: 2.101366]
[Epoch 8/2000] [loss_q: 0.079888] [loss_v: 0.102067] [loss_fd: 0.019087] [total_loss: 0.185772] 

[Epoch 72/2000] [loss_q: 0.026331] [loss_v: 0.031323] [loss_fd: 0.019129] [total_loss: 0.061479] [time: 9.539037]
[Epoch 73/2000] [loss_q: 0.022819] [loss_v: 0.026666] [loss_fd: 0.014261] [total_loss: 0.052338] [time: 9.630102]
[Epoch 74/2000] [loss_q: 0.022413] [loss_v: 0.030041] [loss_fd: 0.017970] [total_loss: 0.056048] [time: 9.717164]
[Epoch 75/2000] [loss_q: 0.020681] [loss_v: 0.025292] [loss_fd: 0.010609] [total_loss: 0.048095] [time: 9.807228]
[Epoch 76/2000] [loss_q: 0.024809] [loss_v: 0.027346] [loss_fd: 0.016024] [total_loss: 0.055361] [time: 9.898293]
[Epoch 77/2000] [loss_q: 0.027071] [loss_v: 0.031478] [loss_fd: 0.014984] [total_loss: 0.061546] [time: 9.988357]
[Epoch 78/2000] [loss_q: 0.022301] [loss_v: 0.029087] [loss_fd: 0.017328] [total_loss: 0.054853] [time: 10.081423]
[Epoch 79/2000] [loss_q: 0.021664] [loss_v: 0.026358] [loss_fd: 0.018976] [total_loss: 0.051817] [time: 10.167485]
[Epoch 80/2000] [loss_q: 0.021744] [loss_v: 0.023419] [loss_fd: 0.014821] [total_loss:

[Epoch 143/2000] [loss_q: 0.018484] [loss_v: 0.018860] [loss_fd: 0.013212] [total_loss: 0.039986] [time: 15.915557]
[Epoch 144/2000] [loss_q: 0.021721] [loss_v: 0.020793] [loss_fd: 0.017467] [total_loss: 0.046007] [time: 16.003619]
[Epoch 145/2000] [loss_q: 0.023096] [loss_v: 0.018944] [loss_fd: 0.012958] [total_loss: 0.044632] [time: 16.091683]
[Epoch 146/2000] [loss_q: 0.017273] [loss_v: 0.020615] [loss_fd: 0.017671] [total_loss: 0.041422] [time: 16.183748]
[Epoch 147/2000] [loss_q: 0.019101] [loss_v: 0.017457] [loss_fd: 0.015389] [total_loss: 0.039636] [time: 16.272811]
[Epoch 148/2000] [loss_q: 0.021491] [loss_v: 0.019618] [loss_fd: 0.014387] [total_loss: 0.043987] [time: 16.359873]
[Epoch 149/2000] [loss_q: 0.020448] [loss_v: 0.018102] [loss_fd: 0.012141] [total_loss: 0.040978] [time: 16.454941]
[Epoch 150/2000] [loss_q: 0.022577] [loss_v: 0.019836] [loss_fd: 0.013608] [total_loss: 0.045134] [time: 16.548007]
[Epoch 151/2000] [loss_q: 0.019122] [loss_v: 0.020331] [loss_fd: 0.01087

[Epoch 215/2000] [loss_q: 0.018282] [loss_v: 0.017785] [loss_fd: 0.011874] [total_loss: 0.038442] [time: 22.384161]
[Epoch 216/2000] [loss_q: 0.019283] [loss_v: 0.014716] [loss_fd: 0.012705] [total_loss: 0.036540] [time: 22.473350]
[Epoch 217/2000] [loss_q: 0.017893] [loss_v: 0.016695] [loss_fd: 0.015461] [total_loss: 0.037680] [time: 22.560412]
[Epoch 218/2000] [loss_q: 0.017719] [loss_v: 0.018266] [loss_fd: 0.016435] [total_loss: 0.039271] [time: 22.648475]
[Epoch 219/2000] [loss_q: 0.019026] [loss_v: 0.016867] [loss_fd: 0.013424] [total_loss: 0.038578] [time: 22.734536]
[Epoch 220/2000] [loss_q: 0.017607] [loss_v: 0.014702] [loss_fd: 0.017729] [total_loss: 0.035855] [time: 22.822599]
[Epoch 221/2000] [loss_q: 0.014907] [loss_v: 0.014505] [loss_fd: 0.014109] [total_loss: 0.032234] [time: 22.913664]
[Epoch 222/2000] [loss_q: 0.018980] [loss_v: 0.014673] [loss_fd: 0.014675] [total_loss: 0.036588] [time: 23.002726]
[Epoch 223/2000] [loss_q: 0.018301] [loss_v: 0.017565] [loss_fd: 0.01522

[Epoch 286/2000] [loss_q: 0.017323] [loss_v: 0.014078] [loss_fd: 0.014718] [total_loss: 0.034344] [time: 28.703100]
[Epoch 287/2000] [loss_q: 0.014468] [loss_v: 0.014228] [loss_fd: 0.013521] [total_loss: 0.031400] [time: 28.795166]
[Epoch 288/2000] [loss_q: 0.016251] [loss_v: 0.013496] [loss_fd: 0.011374] [total_loss: 0.032021] [time: 28.883228]
[Epoch 289/2000] [loss_q: 0.014982] [loss_v: 0.013124] [loss_fd: 0.014275] [total_loss: 0.030961] [time: 28.978296]
[Epoch 290/2000] [loss_q: 0.018678] [loss_v: 0.015721] [loss_fd: 0.018680] [total_loss: 0.038135] [time: 29.066359]
[Epoch 291/2000] [loss_q: 0.015676] [loss_v: 0.012992] [loss_fd: 0.018015] [total_loss: 0.032271] [time: 29.158424]
[Epoch 292/2000] [loss_q: 0.015528] [loss_v: 0.012827] [loss_fd: 0.012590] [total_loss: 0.030872] [time: 29.249489]
[Epoch 293/2000] [loss_q: 0.014813] [loss_v: 0.012582] [loss_fd: 0.013756] [total_loss: 0.030146] [time: 29.336551]
[Epoch 294/2000] [loss_q: 0.015805] [loss_v: 0.015016] [loss_fd: 0.01459

...

[Epoch 1714/2000] [loss_q: 0.005025] [loss_v: 0.002915] [loss_fd: 0.011188] [total_loss: 0.010178] [time: 158.523470]
[Epoch 1715/2000] [loss_q: 0.004955] [loss_v: 0.004229] [loss_fd: 0.015843] [total_loss: 0.012353] [time: 158.613534]
[Epoch 1716/2000] [loss_q: 0.005820] [loss_v: 0.003888] [loss_fd: 0.011480] [total_loss: 0.012004] [time: 158.702598]
[Epoch 1717/2000] [loss_q: 0.005172] [loss_v: 0.004577] [loss_fd: 0.019009] [total_loss: 0.013551] [time: 158.794663]
[Epoch 1718/2000] [loss_q: 0.004417] [loss_v: 0.004443] [loss_fd: 0.015901] [total_loss: 0.012040] [time: 158.884727]
[Epoch 1719/2000] [loss_q: 0.004343] [loss_v: 0.004068] [loss_fd: 0.013589] [total_loss: 0.011129] [time: 158.972790]
[Epoch 1720/2000] [loss_q: 0.004834] [loss_v: 0.004836] [loss_fd: 0.017881] [total_loss: 0.013247] [time: 159.064855]
[Epoch 1721/2000] [loss_q: 0.006228] [loss_v: 0.003738] [loss_fd: 0.012397] [total_loss: 0.012445] [time: 159.153919]
[Epoch 1722/2000] [loss_q: 0.004544] [loss_v: 0.003553] 

[Epoch 1786/2000] [loss_q: 0.004751] [loss_v: 0.004837] [loss_fd: 0.013400] [total_loss: 0.012269] [time: 165.094151]
[Epoch 1787/2000] [loss_q: 0.005287] [loss_v: 0.005637] [loss_fd: 0.014498] [total_loss: 0.013824] [time: 165.187218]
[Epoch 1788/2000] [loss_q: 0.004324] [loss_v: 0.005088] [loss_fd: 0.014809] [total_loss: 0.012374] [time: 165.279283]
[Epoch 1789/2000] [loss_q: 0.005386] [loss_v: 0.005113] [loss_fd: 0.014114] [total_loss: 0.013322] [time: 165.370348]
[Epoch 1790/2000] [loss_q: 0.005344] [loss_v: 0.004750] [loss_fd: 0.013438] [total_loss: 0.012782] [time: 165.458411]
[Epoch 1791/2000] [loss_q: 0.005045] [loss_v: 0.004543] [loss_fd: 0.015305] [total_loss: 0.012650] [time: 165.550476]
[Epoch 1792/2000] [loss_q: 0.005924] [loss_v: 0.005277] [loss_fd: 0.016857] [total_loss: 0.014573] [time: 165.640540]
[Epoch 1793/2000] [loss_q: 0.006254] [loss_v: 0.005119] [loss_fd: 0.015520] [total_loss: 0.014478] [time: 165.731605]
[Epoch 1794/2000] [loss_q: 0.004797] [loss_v: 0.004409] 

[Epoch 1857/2000] [loss_q: 0.003895] [loss_v: 0.003543] [loss_fd: 0.014132] [total_loss: 0.010264] [time: 171.547531]
[Epoch 1858/2000] [loss_q: 0.004445] [loss_v: 0.003799] [loss_fd: 0.013180] [total_loss: 0.010880] [time: 171.638596]
[Epoch 1859/2000] [loss_q: 0.004326] [loss_v: 0.004196] [loss_fd: 0.015770] [total_loss: 0.011677] [time: 171.727659]
[Epoch 1860/2000] [loss_q: 0.003649] [loss_v: 0.003292] [loss_fd: 0.013907] [total_loss: 0.009723] [time: 171.819725]
[Epoch 1861/2000] [loss_q: 0.004864] [loss_v: 0.003169] [loss_fd: 0.012259] [total_loss: 0.010485] [time: 171.908788]
[Epoch 1862/2000] [loss_q: 0.004835] [loss_v: 0.004343] [loss_fd: 0.013962] [total_loss: 0.011970] [time: 172.004857]
[Epoch 1863/2000] [loss_q: 0.003505] [loss_v: 0.003883] [loss_fd: 0.015340] [total_loss: 0.010456] [time: 172.096922]
[Epoch 1864/2000] [loss_q: 0.004039] [loss_v: 0.003681] [loss_fd: 0.011774] [total_loss: 0.010074] [time: 172.186986]
[Epoch 1865/2000] [loss_q: 0.003813] [loss_v: 0.003933] 

[Epoch 1928/2000] [loss_q: 0.003014] [loss_v: 0.003816] [loss_fd: 0.017213] [total_loss: 0.010273] [time: 178.031141]
[Epoch 1929/2000] [loss_q: 0.003688] [loss_v: 0.004458] [loss_fd: 0.011805] [total_loss: 0.010508] [time: 178.119204]
[Epoch 1930/2000] [loss_q: 0.003532] [loss_v: 0.003434] [loss_fd: 0.015351] [total_loss: 0.010037] [time: 178.209268]
[Epoch 1931/2000] [loss_q: 0.004172] [loss_v: 0.004082] [loss_fd: 0.014967] [total_loss: 0.011247] [time: 178.303335]
[Epoch 1932/2000] [loss_q: 0.003873] [loss_v: 0.003442] [loss_fd: 0.012723] [total_loss: 0.009860] [time: 178.391397]
[Epoch 1933/2000] [loss_q: 0.004361] [loss_v: 0.004804] [loss_fd: 0.016742] [total_loss: 0.012514] [time: 178.479460]
[Epoch 1934/2000] [loss_q: 0.003763] [loss_v: 0.003589] [loss_fd: 0.011234] [total_loss: 0.009599] [time: 178.564639]
[Epoch 1935/2000] [loss_q: 0.003459] [loss_v: 0.004381] [loss_fd: 0.014620] [total_loss: 0.010764] [time: 178.652702]
[Epoch 1936/2000] [loss_q: 0.003942] [loss_v: 0.004390] 

[Epoch 1998/2000] [loss_q: 0.003929] [loss_v: 0.003931] [loss_fd: 0.017261] [total_loss: 0.011311] [time: 184.334018]
[Epoch 1999/2000] [loss_q: 0.004133] [loss_v: 0.003862] [loss_fd: 0.013618] [total_loss: 0.010719] [time: 184.425083]


In [42]:
train(n_epochs = n_epochs, batch_size = batch_size, sample_size = sample_size)

[Epoch 0/2000] [loss_q: 0.260088] [loss_v: 0.332011] [loss_fd: 0.060569] [total_loss: 0.604214] [time: 0.362348]
[Epoch 1/2000] [loss_q: 0.287503] [loss_v: 0.326154] [loss_fd: 0.044160] [total_loss: 0.622489] [time: 0.609259]
[Epoch 2/2000] [loss_q: 0.131112] [loss_v: 0.203417] [loss_fd: 0.017466] [total_loss: 0.338021] [time: 0.852190]
[Epoch 3/2000] [loss_q: 0.144157] [loss_v: 0.221039] [loss_fd: 0.012243] [total_loss: 0.367645] [time: 1.085162]
[Epoch 4/2000] [loss_q: 0.118638] [loss_v: 0.133378] [loss_fd: 0.012329] [total_loss: 0.254481] [time: 1.354974]
[Epoch 5/2000] [loss_q: 0.098102] [loss_v: 0.139406] [loss_fd: 0.026695] [total_loss: 0.242847] [time: 1.600890]
[Epoch 6/2000] [loss_q: 0.086333] [loss_v: 0.159048] [loss_fd: 0.026302] [total_loss: 0.250642] [time: 1.836850]
[Epoch 7/2000] [loss_q: 0.094234] [loss_v: 0.132233] [loss_fd: 0.017533] [total_loss: 0.229973] [time: 2.079790]
[Epoch 8/2000] [loss_q: 0.094396] [loss_v: 0.111619] [loss_fd: 0.012599] [total_loss: 0.208534] 

[Epoch 72/2000] [loss_q: 0.024642] [loss_v: 0.034854] [loss_fd: 0.016367] [total_loss: 0.062769] [time: 18.077296]
[Epoch 73/2000] [loss_q: 0.026811] [loss_v: 0.031673] [loss_fd: 0.012733] [total_loss: 0.061030] [time: 18.326199]
[Epoch 74/2000] [loss_q: 0.022409] [loss_v: 0.025794] [loss_fd: 0.012237] [total_loss: 0.050650] [time: 18.551208]
[Epoch 75/2000] [loss_q: 0.024284] [loss_v: 0.025794] [loss_fd: 0.014246] [total_loss: 0.052928] [time: 18.784181]
[Epoch 76/2000] [loss_q: 0.025074] [loss_v: 0.025853] [loss_fd: 0.014932] [total_loss: 0.053913] [time: 19.016159]
[Epoch 77/2000] [loss_q: 0.024607] [loss_v: 0.029681] [loss_fd: 0.021602] [total_loss: 0.058608] [time: 19.254111]
[Epoch 78/2000] [loss_q: 0.023516] [loss_v: 0.027422] [loss_fd: 0.017015] [total_loss: 0.054342] [time: 19.512970]
[Epoch 79/2000] [loss_q: 0.024723] [loss_v: 0.029174] [loss_fd: 0.020100] [total_loss: 0.057917] [time: 19.734992]
[Epoch 80/2000] [loss_q: 0.022025] [loss_v: 0.026754] [loss_fd: 0.016415] [total

[Epoch 143/2000] [loss_q: 0.020937] [loss_v: 0.021366] [loss_fd: 0.013151] [total_loss: 0.044934] [time: 35.119210]
[Epoch 144/2000] [loss_q: 0.020411] [loss_v: 0.021055] [loss_fd: 0.013575] [total_loss: 0.044181] [time: 35.351187]
[Epoch 145/2000] [loss_q: 0.019727] [loss_v: 0.021133] [loss_fd: 0.013258] [total_loss: 0.043511] [time: 35.590135]
[Epoch 146/2000] [loss_q: 0.024310] [loss_v: 0.022181] [loss_fd: 0.012205] [total_loss: 0.048932] [time: 35.822113]
[Epoch 147/2000] [loss_q: 0.021359] [loss_v: 0.017954] [loss_fd: 0.014932] [total_loss: 0.042299] [time: 36.063051]
[Epoch 148/2000] [loss_q: 0.020351] [loss_v: 0.019576] [loss_fd: 0.009641] [total_loss: 0.041854] [time: 36.311954]
[Epoch 149/2000] [loss_q: 0.020758] [loss_v: 0.025812] [loss_fd: 0.012895] [total_loss: 0.049149] [time: 36.578778]
[Epoch 150/2000] [loss_q: 0.021709] [loss_v: 0.020420] [loss_fd: 0.016858] [total_loss: 0.045501] [time: 36.817726]
[Epoch 151/2000] [loss_q: 0.024379] [loss_v: 0.023013] [loss_fd: 0.01688

[Epoch 214/2000] [loss_q: 0.018437] [loss_v: 0.017590] [loss_fd: 0.015192] [total_loss: 0.039065] [time: 53.258289]
[Epoch 215/2000] [loss_q: 0.024173] [loss_v: 0.022344] [loss_fd: 0.014371] [total_loss: 0.049392] [time: 53.507192]
[Epoch 216/2000] [loss_q: 0.020867] [loss_v: 0.020686] [loss_fd: 0.014387] [total_loss: 0.044430] [time: 53.761074]
[Epoch 217/2000] [loss_q: 0.019568] [loss_v: 0.015303] [loss_fd: 0.013982] [total_loss: 0.037668] [time: 53.984091]
[Epoch 218/2000] [loss_q: 0.021069] [loss_v: 0.017405] [loss_fd: 0.014171] [total_loss: 0.041308] [time: 54.241955]
[Epoch 219/2000] [loss_q: 0.019275] [loss_v: 0.017251] [loss_fd: 0.013935] [total_loss: 0.039313] [time: 54.482893]
[Epoch 220/2000] [loss_q: 0.020113] [loss_v: 0.015876] [loss_fd: 0.013780] [total_loss: 0.038745] [time: 54.729805]
[Epoch 221/2000] [loss_q: 0.018738] [loss_v: 0.015447] [loss_fd: 0.016278] [total_loss: 0.037441] [time: 54.963775]
[Epoch 222/2000] [loss_q: 0.019192] [loss_v: 0.018069] [loss_fd: 0.01507

[Epoch 285/2000] [loss_q: 0.015958] [loss_v: 0.014529] [loss_fd: 0.011283] [total_loss: 0.032743] [time: 70.312150]
[Epoch 286/2000] [loss_q: 0.017521] [loss_v: 0.013502] [loss_fd: 0.010312] [total_loss: 0.033086] [time: 70.561054]
[Epoch 287/2000] [loss_q: 0.015732] [loss_v: 0.011536] [loss_fd: 0.010266] [total_loss: 0.029321] [time: 70.787058]
[Epoch 288/2000] [loss_q: 0.015897] [loss_v: 0.014299] [loss_fd: 0.011611] [total_loss: 0.032518] [time: 71.031979]
[Epoch 289/2000] [loss_q: 0.016181] [loss_v: 0.014358] [loss_fd: 0.018039] [total_loss: 0.034147] [time: 71.260970]
[Epoch 290/2000] [loss_q: 0.016252] [loss_v: 0.014678] [loss_fd: 0.017008] [total_loss: 0.034332] [time: 71.490956]
[Epoch 291/2000] [loss_q: 0.020440] [loss_v: 0.015277] [loss_fd: 0.013865] [total_loss: 0.038490] [time: 71.734882]
[Epoch 292/2000] [loss_q: 0.017229] [loss_v: 0.017251] [loss_fd: 0.014890] [total_loss: 0.037458] [time: 71.986772]
[Epoch 293/2000] [loss_q: 0.019801] [loss_v: 0.013779] [loss_fd: 0.01543

...

[Epoch 1695/2000] [loss_q: 0.004611] [loss_v: 0.006010] [loss_fd: 0.018664] [total_loss: 0.014354] [time: 416.140440]
[Epoch 1696/2000] [loss_q: 0.004006] [loss_v: 0.004970] [loss_fd: 0.014860] [total_loss: 0.011948] [time: 416.379388]
[Epoch 1697/2000] [loss_q: 0.005492] [loss_v: 0.005030] [loss_fd: 0.015561] [total_loss: 0.013634] [time: 416.608379]
[Epoch 1698/2000] [loss_q: 0.004578] [loss_v: 0.005091] [loss_fd: 0.012560] [total_loss: 0.012181] [time: 416.831396]
[Epoch 1699/2000] [loss_q: 0.005022] [loss_v: 0.007558] [loss_fd: 0.015174] [total_loss: 0.015615] [time: 417.054414]
[Epoch 1700/2000] [loss_q: 0.002870] [loss_v: 0.004029] [loss_fd: 0.011196] [total_loss: 0.009138] [time: 417.291369]
[Epoch 1701/2000] [loss_q: 0.005038] [loss_v: 0.005939] [loss_fd: 0.017121] [total_loss: 0.014401] [time: 417.550229]
[Epoch 1702/2000] [loss_q: 0.003433] [loss_v: 0.005039] [loss_fd: 0.013697] [total_loss: 0.011211] [time: 417.791167]
[Epoch 1703/2000] [loss_q: 0.004213] [loss_v: 0.004938] 

[Epoch 1765/2000] [loss_q: 0.004135] [loss_v: 0.008292] [loss_fd: 0.017323] [total_loss: 0.015892] [time: 433.295854]
[Epoch 1766/2000] [loss_q: 0.003364] [loss_v: 0.006041] [loss_fd: 0.012239] [total_loss: 0.011853] [time: 433.546749]
[Epoch 1767/2000] [loss_q: 0.002218] [loss_v: 0.009996] [loss_fd: 0.012213] [total_loss: 0.014656] [time: 433.780718]
[Epoch 1768/2000] [loss_q: 0.003520] [loss_v: 0.025459] [loss_fd: 0.019381] [total_loss: 0.032855] [time: 434.044556]
[Epoch 1769/2000] [loss_q: 0.002924] [loss_v: 0.008064] [loss_fd: 0.013053] [total_loss: 0.013598] [time: 434.290472]
[Epoch 1770/2000] [loss_q: 0.003768] [loss_v: 0.018284] [loss_fd: 0.019896] [total_loss: 0.026032] [time: 434.540371]
[Epoch 1771/2000] [loss_q: 0.003670] [loss_v: 0.008330] [loss_fd: 0.016035] [total_loss: 0.015207] [time: 434.765379]
[Epoch 1772/2000] [loss_q: 0.003218] [loss_v: 0.013641] [loss_fd: 0.014859] [total_loss: 0.019831] [time: 435.020257]
[Epoch 1773/2000] [loss_q: 0.004918] [loss_v: 0.014556] 

[Epoch 1835/2000] [loss_q: 0.003882] [loss_v: 0.008795] [loss_fd: 0.018171] [total_loss: 0.016312] [time: 451.379180]
[Epoch 1836/2000] [loss_q: 0.004267] [loss_v: 0.007541] [loss_fd: 0.016713] [total_loss: 0.015150] [time: 451.611158]
[Epoch 1837/2000] [loss_q: 0.003166] [loss_v: 0.007025] [loss_fd: 0.013354] [total_loss: 0.012862] [time: 451.852096]
[Epoch 1838/2000] [loss_q: 0.003705] [loss_v: 0.007984] [loss_fd: 0.016133] [total_loss: 0.014915] [time: 452.087060]
[Epoch 1839/2000] [loss_q: 0.003087] [loss_v: 0.007106] [loss_fd: 0.012750] [total_loss: 0.012743] [time: 452.314060]
[Epoch 1840/2000] [loss_q: 0.003722] [loss_v: 0.007452] [loss_fd: 0.014147] [total_loss: 0.014003] [time: 452.575907]
[Epoch 1841/2000] [loss_q: 0.003271] [loss_v: 0.008253] [loss_fd: 0.015794] [total_loss: 0.014682] [time: 452.807884]
[Epoch 1842/2000] [loss_q: 0.003359] [loss_v: 0.007416] [loss_fd: 0.014312] [total_loss: 0.013637] [time: 453.039862]
[Epoch 1843/2000] [loss_q: 0.002890] [loss_v: 0.008109] 

[Epoch 1905/2000] [loss_q: 0.003332] [loss_v: 0.005040] [loss_fd: 0.014141] [total_loss: 0.011200] [time: 469.023443]
[Epoch 1906/2000] [loss_q: 0.003856] [loss_v: 0.006829] [loss_fd: 0.017258] [total_loss: 0.014136] [time: 469.280308]
[Epoch 1907/2000] [loss_q: 0.003295] [loss_v: 0.005196] [loss_fd: 0.014383] [total_loss: 0.011367] [time: 469.546143]
[Epoch 1908/2000] [loss_q: 0.003924] [loss_v: 0.005561] [loss_fd: 0.014324] [total_loss: 0.012350] [time: 469.795039]
[Epoch 1909/2000] [loss_q: 0.004388] [loss_v: 0.005894] [loss_fd: 0.016807] [total_loss: 0.013643] [time: 470.024031]
[Epoch 1910/2000] [loss_q: 0.005236] [loss_v: 0.006700] [loss_fd: 0.015464] [total_loss: 0.015029] [time: 470.263976]
[Epoch 1911/2000] [loss_q: 0.003339] [loss_v: 0.005405] [loss_fd: 0.013419] [total_loss: 0.011428] [time: 470.523829]
[Epoch 1912/2000] [loss_q: 0.005367] [loss_v: 0.005625] [loss_fd: 0.012773] [total_loss: 0.013547] [time: 470.778706]
[Epoch 1913/2000] [loss_q: 0.004291] [loss_v: 0.005792] 

[Epoch 1975/2000] [loss_q: 0.004433] [loss_v: 0.004585] [loss_fd: 0.014834] [total_loss: 0.011984] [time: 486.406848]
[Epoch 1976/2000] [loss_q: 0.003058] [loss_v: 0.004198] [loss_fd: 0.014719] [total_loss: 0.010200] [time: 486.633850]
[Epoch 1977/2000] [loss_q: 0.003636] [loss_v: 0.003970] [loss_fd: 0.012028] [total_loss: 0.010011] [time: 486.871800]
[Epoch 1978/2000] [loss_q: 0.004546] [loss_v: 0.005344] [loss_fd: 0.016754] [total_loss: 0.013241] [time: 487.101787]
[Epoch 1979/2000] [loss_q: 0.005041] [loss_v: 0.005304] [loss_fd: 0.012742] [total_loss: 0.012894] [time: 487.341730]
[Epoch 1980/2000] [loss_q: 0.003467] [loss_v: 0.004620] [loss_fd: 0.011756] [total_loss: 0.010438] [time: 487.592624]
[Epoch 1981/2000] [loss_q: 0.004559] [loss_v: 0.004587] [loss_fd: 0.011237] [total_loss: 0.011393] [time: 487.844514]
[Epoch 1982/2000] [loss_q: 0.003550] [loss_v: 0.004643] [loss_fd: 0.014796] [total_loss: 0.011152] [time: 488.066536]
[Epoch 1983/2000] [loss_q: 0.003122] [loss_v: 0.005511] 

In [35]:
test_pred_q0 = piml_q(test_input_q, adj_n, adj_correlation_q, adj_knn_q)
test_pred_q = r_norm(test_pred_q0, max_q, min_q)

test_pred_v0 = piml_v(test_input_v, adj_n, adj_correlation_v, adj_knn_v)
test_pred_v = r_norm(test_pred_v0, max_v, min_v)

In [36]:
print(nn_loss(test_pred_q, test_true_q)**0.5)
print(nn_loss(test_pred_v, test_true_v)**0.5)

l1_loss = nn.L1Loss()
print(l1_loss(test_pred_q, test_true_q))
print(l1_loss(test_pred_v, test_true_v))

mape_q = torch.abs(test_pred_q-test_true_q)/test_true_q
mape_q = mape_q[mape_q<1000]

mape_v = torch.abs(test_pred_v-test_true_v)/test_true_v
mape_v = mape_v[mape_v<1000]

print(torch.mean(mape_q))
print(torch.mean(mape_v))

tensor(256.7314, grad_fn=<PowBackward0>)
tensor(7.0437, grad_fn=<PowBackward0>)
tensor(199.2998, grad_fn=<MeanBackward0>)
tensor(5.0241, grad_fn=<MeanBackward0>)
tensor(0.4800, grad_fn=<MeanBackward0>)
tensor(0.0770, grad_fn=<MeanBackward0>)
